In [1]:
import folium
from folium import Map, Marker

import json

from pprint import pprint

from openrouteservice import client
from openrouteservice.directions import directions

from shapely import geometry, wkt
from shapely.geometry import Point, LineString

## Rostock construction sites

In [2]:
api_key = '58d904a497c67e00015b45fc2a6b6872037d44119582ef40cdf264c4' # individual api key
clnt = client.Client(key=api_key) # creating client with api key
map3 = folium.Map(tiles='Stamen Toner', location=([54.13207, 12.101612]), zoom_start=12) 

with open('baustellen_rostock.json') as l: # Load in construction data points
    rostock_json = json.load(l)
    
construction_points = []
for poly in rostock_json['features']:
    poly_coords2 = poly['geometry']['coordinates']
    folium.features.Marker(list(reversed(poly_coords2)), popup='Construction<br>{0}'.format(poly_coords2)).add_to(map3)
    construction_points.append(poly_coords2)
    
map3

The number of construction sites is too large for the GET direction service.

In [3]:
def create_route(avoided_point_list, n=0):
    route_request = {'coordinates': [start, destination], 
                    'format_out': 'geojson',
                    'profile': 'driving-car',
                    'preference': 'shortest',
                      'instructions': False,
                     'options': {'avoid_polygons': {'type': 'MultiPolygon', 'coordinates': avoided_point_list}}
                     }
    route_directions = clnt.directions(**route_request)
    
    return route_directions


def create_buffer(route_directions):
    line_tup = []
    for line in route_directions['features'][0]['geometry']['coordinates']:
        line_tup_row = tuple(line)
        line_tup.append(line_tup_row)  

    linestring = LineString(line_tup)
    dilated_route = linestring.buffer(0.001)
    dilated_exter = list(dilated_route.exterior.coords)

    for n in dilated_exter:
        di_list = list(n)
        
    return dilated_route

In [4]:
start = [12.088019, 54.084592]
destination = [12.131739, 54.082398] # First example: Route available.
#destination = [12.129453, 54.082845] # Second example: No route available because of too many construction sites.

# Request route without avoiding any construction points
avoided_point_list = []
route_directions = create_route(avoided_point_list) 
dilated_route = create_buffer(route_directions)
    
# Create geometry of construction points 
point_geom = []
for point in construction_points:
    poi_tup = tuple(point)
    poi_geom = geometry.Point(poi_tup)
    point_geom.append(poi_geom)

try:
    # Request route with avoiding construction points which are located on route    
    for point in point_geom:
        if point.within(dilated_route): 
            point_coords = list(point.coords) # get point coordinates

            # Create avoiding area around point
            avoid_box = []
            t = 0.0003
            for x,y in point_coords: 
                point_area = [[x-t, y-t], [x+t, y-t], [x+t, y+t], [x-t, y+t], [x-t, y-t]]
                avoid_box.append(point_area) 

            # Data is needed as strings in listed lists 
            poly_coords_list = [] # Coords of each polygon -> will be deleted after use
            for poly in avoid_box:
                for row in poly:
                    coords = list(row)
                    i = 0
                    for l in coords:
                        i = i+1
                        if i%2 != 0:
                            x = l
                        if i%2 == 0:
                            y = l

                            kombi_coords = [str(x), str(y)]
                            poly_coords_list.append(kombi_coords)

                avoided_point_list.append([poly_coords_list]) # Coords of all related polygons
                poly_coords_list = []

            # Create new route
            route_directions = create_route(avoided_point_list, 1)
            dilated_route = create_buffer(route_directions)
        
except Exception: print('Sorry, there is no route available to the requested destination because of too may construction sites. Please choose a nearby location for a new routing request or park the car in the aim area and walk the last part.')    

route_coords = [(y,x) for x,y in route_directions['features'][0]['geometry']['coordinates']]
folium.PolyLine(route_coords, color='red').add_to(map3)

folium.map.Marker(list(reversed(start)), folium.Icon(color='green')).add_to(map3)
folium.map.Marker(list(reversed(destination)), folium.Icon(color='green')).add_to(map3)

map3